In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset, Dataset
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import accuracy_score
from google.colab import files

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##### **Load Datasets**

In [2]:
def upload_dataset_from_system():
  uploaded = files.upload()
  data = Dataset.from_pandas(pd.read_parquet(list(uploaded.keys())[0]))
  return data

# --- return dataset classes ---
def dataset_classes(dataset):
  return list(set([item['label'] for item in dataset]))

# --- model name
model_name = "distilbert-base-uncased"

# --- tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
# --- load dataset 20-classes ---
train_data = upload_dataset_from_system()
val_data = upload_dataset_from_system()
test_data = upload_dataset_from_system()  # max_length -> 40

Saving train-00000-of-00001.parquet to train-00000-of-00001.parquet


Saving validation-00000-of-00001.parquet to validation-00000-of-00001.parquet


Saving test-00000-of-00001.parquet to test-00000-of-00001.parquet


In [3]:
# --- load sst5 ---
sst = load_dataset("SetFit/sst5")  # max_length -> 60

README.md:   0%|          | 0.00/421 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.jsonl: 0.00B [00:00, ?B/s]

dev.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/8544 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1101 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2210 [00:00<?, ? examples/s]

##### **preprocess & Dataset-utilities**

In [5]:

def preproces_data(example,max_len,text ='text',label = 'label'):
  tokenized_item = tokenizer(example[text],truncation=True,max_length=max_len,padding="max_length")
  inp_ids = tokenized_item['input_ids']
  attn_msk = tokenized_item['attention_mask']
  label = example[label]

  return {
         "input_ids": inp_ids, # Convert list to tensor
          "attention_mask": attn_msk, # Convert list to tensor
          "labels": label # Return the original label index
    }

In [6]:
# --- 20 classes dataset ---
train_dataset = train_data.map(preproces_data, fn_kwargs={'max_len': 40}, remove_columns=['text','label'])
val_dataset = val_data.map(preproces_data, fn_kwargs={'max_len': 40}, remove_columns=['text','label'])
test_dataset = test_data.map(preproces_data, fn_kwargs={'max_len': 40}, remove_columns=['text','label'])

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
# --- preprocess sst5 ---
sst_train = sst['train'].map(preproces_data,fn_kwargs={'max_len': 60},remove_columns=['text','label','label_text'])
sst_val = sst['validation'].map(preproces_data,fn_kwargs={'max_len': 60},remove_columns=['text','label','label_text'])
sst_test = sst['test'].map(preproces_data,fn_kwargs={'max_len': 60},remove_columns=['text','label','label_text'])

Map:   0%|          | 0/8544 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

Map:   0%|          | 0/2210 [00:00<?, ? examples/s]

In [8]:
def collate_fn(batch):
    input_ids = [
        torch.tensor(item['input_ids'], dtype=torch.long)
        if not isinstance(item['input_ids'], torch.Tensor) else item['input_ids'].long()
        for item in batch
    ]
    attention_mask = [
        torch.tensor(item['attention_mask'], dtype=torch.long)
        if not isinstance(item['attention_mask'], torch.Tensor) else item['attention_mask'].long()
        for item in batch
    ]

    labels = torch.stack([
        torch.tensor(item['labels'], dtype=torch.long) # Changed to torch.long
        if not isinstance(item['labels'], torch.Tensor) else item['labels'].long()
        for item in batch
    ])


    input_ids_padded = pad_sequence(
        input_ids, batch_first=True, padding_value=tokenizer.pad_token_id
    )
    attention_mask_padded = pad_sequence(
        attention_mask, batch_first=True, padding_value=0
    )

    return {
        "input_ids": input_ids_padded,        # (batch_size, seq_len)
        "attention_mask": attention_mask_padded,  # (batch_size, seq_len)
        "labels": labels                      # (batch_size, num_classes)
    }

##### **train , test methods**

In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
    }

In [10]:
def FineTune(model,train_data,val_data,num_epochs,batch_size,lr):

  training_args = TrainingArguments(
      output_dir='./results',
      num_train_epochs=num_epochs,
      per_device_train_batch_size=batch_size,
      per_device_eval_batch_size=batch_size,
      learning_rate=lr,
      weight_decay=0.01,
      eval_strategy="epoch",
      save_strategy="epoch",
      load_best_model_at_end=True,
      logging_steps=500,
      report_to = "tensorboard"
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_data,
      eval_dataset=val_data,
      compute_metrics=compute_metrics,
      data_collator=collate_fn
  )

  trainer.train()

  return trainer.model

In [11]:
def test_func(model, test_data, batch_size):
    model.eval()
    args = TrainingArguments(
        output_dir='./results',
        per_device_eval_batch_size=batch_size,
        logging_steps=500,
        report_to = "tensorboard"
    )
    trainer = Trainer(
        model=model,
        args=args,
        eval_dataset=test_data,
        compute_metrics=compute_metrics,
        data_collator=collate_fn
    )

    eval_results = trainer.evaluate()
    return eval_results

##### **distilBERT & sst5**

In [12]:
# --- load model ---
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5).to(device)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# --- test before fine-tune
eval_res = test_func(model,sst_test,16)
print(eval_res)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 1.600426435470581, 'eval_model_preparation_time': 0.002, 'eval_accuracy': 0.2823529411764706, 'eval_runtime': 221.5792, 'eval_samples_per_second': 9.974, 'eval_steps_per_second': 0.627}


In [13]:
# --- fine tune
tuned_model = FineTune(model,sst_train,sst_val,3,32,3e-5)  # lr -> 3e-5

# --- test after fine-tune
eval_res = test_func(tuned_model,sst_test,16)
print(eval_res)

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.173328,0.462307
2,1.136700,1.164625,0.484105
3,1.136700,1.206734,0.505904


{'eval_loss': 1.0917497873306274, 'eval_model_preparation_time': 0.0012, 'eval_accuracy': 0.5221719457013575, 'eval_runtime': 3.7979, 'eval_samples_per_second': 581.907, 'eval_steps_per_second': 36.6}


##### **distilBERT & 20-classes**

In [14]:
# --- load model ---
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=20).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# --- test before fine-tune
eval_res = test_func(model,test_dataset,32)
print(eval_res)

{'eval_loss': 3.010295867919922, 'eval_model_preparation_time': 0.0023, 'eval_accuracy': 0.03048, 'eval_runtime': 55.5403, 'eval_samples_per_second': 900.248, 'eval_steps_per_second': 28.142}


In [15]:
# --- fine tune
tuned_model = FineTune(model,train_dataset,val_dataset,3,32,3e-5)

# --- test after fine-tune
eval_res = test_func(tuned_model,test_dataset,32)
print(eval_res)

Epoch,Training Loss,Validation Loss,Accuracy
1,2.292000,2.228643,0.329600
2,2.021400,2.170111,0.352800
3,1.832800,2.184023,0.358400


{'eval_loss': 2.1226062774658203, 'eval_model_preparation_time': 0.0012, 'eval_accuracy': 0.36282, 'eval_runtime': 54.9756, 'eval_samples_per_second': 909.495, 'eval_steps_per_second': 28.431}
